In [3]:
import tensorflow as tf
import numpy as np
import random
from datetime import datetime

In [6]:
class DataGeneration:
    
    def __init__(self, name, file_path, seperation_rate, is_normalized = False):
        self.name = name
        self.file_path = file_path
        self.seperation_rate = seperation_rate
        self.is_normalized = is_normalized
        print("DataGeneration object is created !!")
        
    def data_normalized_using_min_max(self, loaded_data):
        transpose_loaded_data = loaded_data.T
        
        print("transpose_loaded_data.shape = ", transpose_loaded_data.shape)
        
        transpose_normalized_data_list = []
        
        for index in range(len(transpose_loaded_data)):
            max_value = np.max(transpose_loaded_data[index, :])
            min_value = np.min(transpose_loaded_data[index, :])
            transpose_normalized_data_list.append((transpose_loaded_data[index, :] - min_value)/(max_value - min_value))
        
        transpose_normalized_data = np.array(transpose_normalized_data_list)
        print("transpose_normalized_data.shape = ",transpose_normalized_data.shape)
        
        normalized_data = transpose_normalized_data.T
        print("normalized_data.shape = ", normalized_data.shape)
        
        data_save_path = './Normalized_' + self.name + '_data.csv'
        
        np.savetxt(data_save_path, normalized_data, delimiter= ',')
        
        return normalized_data
    
    def generate(self):
        loaded_data = np.loadtxt(self.file_path, delimiter= ',', dtype = np.float32)
        print("loaded_data.shape = ", loaded_data.shape)
        
        if self.is_normalized == True:
            loaded_data = self.data_normalized_using_min_max(loaded_data)
        
        training_data_list = []
        test_data_list = []
        
        total_data_num = len(loaded_data)
        test_data_num = int(len(loaded_data) * self.seperation_rate)
        
        total_data_index_list = [index for index in range(total_data_num)]
        
        random.shuffle(total_data_index_list)
        
        ########################### test data와 training data 나누기 ######################################
        
        test_data_index_list = total_data_index_list[ 0: test_data_num]
        
        print("length of test_data_index_list = ", len(test_data_index_list))
        
        #training data 를 위한 인덱스는 total_data_index_list 에서 test data인덱스를 제외한 나머지 부분
        training_data_index_list = total_data_index_list[ test_data_num : ]
        
        print("length of training_data_index_list = ", len(training_data_index_list))
        
        # training data 구성
        for training_data_index in training_data_index_list:
            training_data_list.append(loaded_data[training_data_index])
        
        # test data 구성
        for test_data_index in test_data_index_list:
            test_data_list.append(loaded_data[test_data_index])
            
        #generate training data from training_data list using np.array(___)
        training_data = np.array(training_data_list)
        
        #generate test data from test_data list using np.array(___)
        test_data = np.array(test_data_list)
        
        #verification shape
        print("training_data.shape = ", training_data.shape)
        print("test_data.shape = ", test_data.shape)
        
        #save training & test data(.csv)
        training_data_save_path = './random_' + self.name + '_training_data.csv'
        test_data_save_path = './random_' + self.name + '_test_data.csv'
        
        np.savetxt(training_data_save_path, training_data, delimiter= ',')
        np.savetxt(test_data_save_path, test_data, delimiter=',')
        
        return training_data, test_data

In [7]:
seperation_rate = 0.3
data_obj = DataGeneration('ThoracicSurgery', './ThoracicSurgery.csv', seperation_rate, True)

(training_data, test_data) = data_obj.generate()

print("training_data.shape = ", training_data.shape)
print("test_data.shape = ", test_data.shape)

DataGeneration object is created !!
loaded_data.shape =  (470, 18)
transpose_loaded_data.shape =  (18, 470)
transpose_normalized_data.shape =  (18, 470)
normalized_data.shape =  (470, 18)
length of test_data_index_list =  141
length of training_data_index_list =  329
training_data.shape =  (329, 18)
test_data.shape =  (141, 18)
training_data.shape =  (329, 18)
test_data.shape =  (141, 18)


In [9]:
training_x_data = training_data[:, 0:-1]
training_t_data = training_data[:, [-1]]

print("training_x_data.shape = ",training_x_data.shape)
print("training_t_data.shape = ",training_t_data.shape)

test_x_data = test_data[:, 0:-1]
test_t_data = test_data[:, [-1]]

print("test_x_data.shape = ", test_x_data.shape)
print("test_t_data.shape = ", test_t_data.shape)

training_x_data.shape =  (329, 17)
training_t_data.shape =  (329, 1)
test_x_data.shape =  (141, 17)
test_t_data.shape =  (141, 1)


In [10]:
learning_rate = 0.01
input_nodes = training_x_data.shape[1]
hidden_nodes = 10
output_nodes = 1
epochs = 1

In [12]:
X = tf.placeholder(tf.float32, [None, input_nodes])
T = tf.placeholder(tf.float32, [None, output_nodes])

W2 = tf.Variable(tf.random_normal([input_nodes, hidden_nodes]))
b2 = tf.Variable(tf.random_normal([hidden_nodes]))

W3 = tf.Variable(tf.random_normal([hidden_nodes, output_nodes]))
b3 = tf.Variable(tf.random_normal([output_nodes]))

In [14]:
Z2 = tf.matmul(X,W2) + b2
A2 = tf.sigmoid(Z2)

Z3 = tf.matmul(A2,W3) + b3
y = A3 = tf.sigmoid(Z3)

loss = -tf.reduce_mean( T * tf.log(y) + (1-T)*tf.log(1-y))

In [15]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [16]:
predicted = tf.cast(y > 0.5, dtype = tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, T), dtype = tf.float32))

In [17]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    start_time = datetime.now()
    
    for i in range(epochs):
        for step in range(10001):
            loss_val, _ = sess.run([loss, train], feed_dict = {X : training_x_data, T : training_t_data})
            
            if step % 500 == 0:
                print("epochs : ", epochs, "step = ", step, "loss_val = ", loss_val)
    end_time = datetime.now()
    print("")
    print("Elapsed Time = ", end_time - start_time)
    
    y_val, predicted_val, accuracy_val = sess.run([y, predicted, accuracy], feed_dict = {X: test_x_data, T: test_t_data})
    
    print("y_val.shape = ", y_val.shape, ", predicted_val.shape = ", predicted_val.shape)
    print("accuracy_val = ", accuracy_val)
    

epochs :  1 step =  0 loss_val =  0.51010954
epochs :  1 step =  500 loss_val =  0.45164222
epochs :  1 step =  1000 loss_val =  0.44919738
epochs :  1 step =  1500 loss_val =  0.44712356
epochs :  1 step =  2000 loss_val =  0.4452011
epochs :  1 step =  2500 loss_val =  0.44341066
epochs :  1 step =  3000 loss_val =  0.44173688
epochs :  1 step =  3500 loss_val =  0.44016594
epochs :  1 step =  4000 loss_val =  0.4386867
epochs :  1 step =  4500 loss_val =  0.43728897
epochs :  1 step =  5000 loss_val =  0.43596452
epochs :  1 step =  5500 loss_val =  0.43470588
epochs :  1 step =  6000 loss_val =  0.43350688
epochs :  1 step =  6500 loss_val =  0.43236172
epochs :  1 step =  7000 loss_val =  0.43126583
epochs :  1 step =  7500 loss_val =  0.4302148
epochs :  1 step =  8000 loss_val =  0.4292049
epochs :  1 step =  8500 loss_val =  0.428233
epochs :  1 step =  9000 loss_val =  0.42729622
epochs :  1 step =  9500 loss_val =  0.42639178
epochs :  1 step =  10000 loss_val =  0.42551753
e

epochs :  1 step =  84500 loss_val =  0.37688273
epochs :  1 step =  85000 loss_val =  0.37664878
epochs :  1 step =  85500 loss_val =  0.37641454
epochs :  1 step =  86000 loss_val =  0.37617984
epochs :  1 step =  86500 loss_val =  0.37594467
epochs :  1 step =  87000 loss_val =  0.37570912
epochs :  1 step =  87500 loss_val =  0.37547332
epochs :  1 step =  88000 loss_val =  0.37523723
epochs :  1 step =  88500 loss_val =  0.3750009
epochs :  1 step =  89000 loss_val =  0.3747643
epochs :  1 step =  89500 loss_val =  0.37452725
epochs :  1 step =  90000 loss_val =  0.37429
epochs :  1 step =  90500 loss_val =  0.37405235
epochs :  1 step =  91000 loss_val =  0.3738146
epochs :  1 step =  91500 loss_val =  0.37357628
epochs :  1 step =  92000 loss_val =  0.37333778
epochs :  1 step =  92500 loss_val =  0.37309912
epochs :  1 step =  93000 loss_val =  0.37286013
epochs :  1 step =  93500 loss_val =  0.3726208
epochs :  1 step =  94000 loss_val =  0.3723811
epochs :  1 step =  94500 lo